### GPU

In [ ]:
!pip install kaggle

In [ ]:
"""
from google.colab import files
files.upload()
"""

'\nfrom google.colab import files\nfiles.upload()\n'

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#!kaggle datasets download -d asaniczka/top-spotify-songs-in-73-countries-daily-updated -p /content

In [ ]:
#!kaggle datasets download -d abdulszz/spotify-most-streamed-songs -p /content

In [ ]:
#!unzip spotify-most-streamed-songs.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

In [ ]:
!pip install yt-dlp

In [ ]:
!ls

 MY_MUSIC.csv   sample_data  'song names.txt'  'song name usa_1.txt'


In [ ]:
def search_youtube(song_name, singer_name):
    """
    Searches YouTube for a song by name and singer, and returns the URL of the top result.

    Parameters:
    - song_name (str): The name of the song.
    - singer_name (str): The name of the singer.

    Returns:
    - str: The URL of the top YouTube search result.
    """
    query = f"{song_name} {singer_name}"
    command = f'yt-dlp "ytsearch:{query}" --get-id --skip-download'

    try:
        # Execute the search and retrieve video ID
        video_id = subprocess.check_output(command, shell=True, text=True).strip()
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"
        return youtube_url
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during YouTube search: {e}")
        return None

In [ ]:
search_youtube("Aaj Ki Raat","Madhubanti Bagchi and Divya Kumar")

'https://www.youtube.com/watch?v=roz9sXFkTuE'

In [ ]:
search_youtube("Cruel Summer", "Taylor Swift")

'https://www.youtube.com/watch?v=ic8j13piAhQ'

In [ ]:
search_youtube("Shape of You","Ed Sheeran")

'https://www.youtube.com/watch?v=JGwWNGJdvx8'

In [ ]:
search_youtube("APT.","ROSE, Bruno Mars")

'https://www.youtube.com/watch?v=8Ebqe2Dbzls'

In [ ]:
search_youtube("運轉人生 - 影集《華麗計程車行》插曲","邱軍")

'https://www.youtube.com/watch?v=x-OinY-8bxc'

In [ ]:
import re
import os
import subprocess

def download_youtube_audio(df_songs, output_folder, audio_format="mp3"):
    """
    Download audio from YouTube videos using URLs from a DataFrame and save them to a specified folder.
    The audio file is saved in the format: songname_(artistname).mp3 for each artist.

    Parameters:
    - df_songs (pd.DataFrame): DataFrame containing songs, their corresponding URLs, and artist names.
    - output_folder (str): The folder where the audio files will be saved.
    - audio_format (str): The audio format for the output files (default is 'mp3').
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Check if the 'url' column is present in the DataFrame
    if 'url' not in df_songs.columns:
        print("The 'url' column is missing from the dataset.")
        return

    # Check if the 'Artists' column is present in the DataFrame
    if 'Artist' not in df_songs.columns:
        print("The 'Artists' column is missing from the dataset.")
        return

    youtube_pattern = re.compile(r'(https?://)?(www\.)?(youtube\.com/watch\?v=|youtu\.be/)([a-zA-Z0-9_-]{11})')
    audio_counter = 1  # Initialize the audio file counter

    for index, row in df_songs.iterrows():
        url = row["url"].strip()
        song_title = row["Song"].strip().replace('/', '-')  # Replace any slashes to avoid path issues
        artist_names = row["Artist"].strip().replace('/', '-').replace(' ', '_')  # Clean up artist names

        # Split multiple artists if there are any
        artists_list = artist_names.split(',')

        if youtube_pattern.match(url):
            for artist in artists_list:
                artist = artist.strip()  # Remove any extra spaces around artist names
                formatted_filename = f"{audio_counter}.{song_title}_({artist})"

                # Create the command to fetch audio
                audio_file_path = f"{output_folder}/{formatted_filename}.{audio_format}"
                command = f'yt-dlp -x --audio-format {audio_format} -o "{audio_file_path}" "{url}"'

                try:
                    subprocess.run(command, shell=True, check=True)
                    print(f"[{audio_counter}] Audio fetched successfully: {formatted_filename} - {url}")
                    audio_counter += 1  # Increment counter only for successful fetch
                except subprocess.CalledProcessError as e:
                    print(f"[{audio_counter}] No audio fetched for: {formatted_filename} - {url} - Error: {e}")
                    # Do not increment audio_counter if fetch fails
        else:
            print(f"[{audio_counter}] No audio fetched for: {song_title} - {url} - Invalid URL")


In [ ]:
#download_youtube_audio("https://www.youtube.com/watch?v=ic8j13piAhQ")

In [ ]:
#download_youtube_audio("https://www.youtube.com/watch?v=JGwWNGJdvx8")

In [ ]:
#download_youtube_audio("https://www.youtube.com/watch?v=8Ebqe2Dbzls")

In [ ]:
from IPython.display import Audio, display
import os


def play_audio(file_path_or_url):
    """
    Play an MP3 audio file from a local path or a URL.

    Parameters:
    - file_path_or_url (str): Path to the MP3 file or a URL of the audio file.
    """
    # Check if the provided path is a local file and exists
    if os.path.isfile(file_path_or_url):
        print(f"Playing local file: {file_path_or_url}")
    else:
        print(f"Playing audio from URL: {file_path_or_url}")

    # Load and play the audio file
    audio = Audio(file_path_or_url, autoplay=True)
    display(audio)

In [ ]:
#play_audio("/content/audio_files/Taylor Swift - Cruel Summer (Official Audio).mp3")

In [ ]:
#play_audio("/content/audio_files/Ed Sheeran - Shape of You (Official Music Video).mp3")

In [ ]:
#play_audio("/content/audio_files/ROSÉ & Bruno Mars - APT. (Official Lyric Video).mp3")

In [ ]:
column_names = ["Artist", "Country"]
df = pd.read_csv("/content/MY_MUSIC.csv",names=column_names,header=None)
df.head(2)

,Artist,Country
0,Taylor Swift,USA
1,Taylor Swift,USA


In [ ]:
len(df)

840

In [ ]:
df["Country"].value_counts()

,count
Country,
USA,440
India,400


In [ ]:
df["Artist"].value_counts()

,count
Artist,
Taylor Swift,20
Shreya Ghoshal,20
Sunidhi Chauhan,20
Sonu Nigam,20
Jubin Nautiyal,20
Pankaj Udas,20
Darshan Raval,20
Diljit Dosanjh,20
Atif Aslam,20


In [ ]:
df_ind =df[df['Country'] == "India"]
df_ind

,Artist,Country
440,Arijit Singh,India
441,Arijit Singh,India
442,Arijit Singh,India
443,Arijit Singh,India
444,Arijit Singh,India
...,...,...
835,Vishal Mishra,India
836,Vishal Mishra,India
837,Vishal Mishra,India
838,Vishal Mishra,India


In [ ]:
df_ind.head(2)

,Artist,Country
440,Arijit Singh,India
441,Arijit Singh,India


In [ ]:
len(df_ind)

400

In [ ]:
def add_songs_to_dataframe(df, txt_file_path, artist_col='Artist', country_col='Country', song_col='Song'):
    # Create a list to hold the new rows
    new_rows = []

    # Read the text file and populate the new rows
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        current_artist = None

        for line in lines:
            line = line.strip()
            if line:  # If the line is not empty
                # Check if the line is an artist name (not enclosed in quotes)
                if not line.startswith('"') and not line.startswith('“'):
                    current_artist = line.strip('“”"')  # Remove any quotes from the artist name
                else:  # This line contains a song
                    # Remove quotes and store the song
                    song = line.strip('“”"')  # Remove quotes from the song title
                    # Add the song as a new row with the corresponding artist
                    if current_artist:
                        new_rows.append({artist_col: current_artist, country_col: None, song_col: song})

    # Create a new DataFrame from the new rows
    df_songs = pd.DataFrame(new_rows)

    return df_songs

In [ ]:
df_ind = add_songs_to_dataframe(df_ind,"/content/song names.txt")
df_ind

,Artist,Country,Song
0,Arijit Singh,None,Kesariya
1,Arijit Singh,None,Tum Hi Ho
2,Arijit Singh,None,Raabta
3,Arijit Singh,None,Channa Mereya
4,Arijit Singh,None,Pal
...,...,...,...
395,Vishal Mishra,None,Tum Hi Aana
396,Vishal Mishra,None,Jab Koi Baat
397,Vishal Mishra,None,Tujhse Naraz Nahi
398,Vishal Mishra,None,Main Wahi Hoon


In [ ]:
len(df_ind)

400

In [ ]:
df_ind

,Artist,Country,Song
0,Arijit Singh,None,Kesariya
1,Arijit Singh,None,Tum Hi Ho
2,Arijit Singh,None,Raabta
3,Arijit Singh,None,Channa Mereya
4,Arijit Singh,None,Pal
...,...,...,...
395,Vishal Mishra,None,Tum Hi Aana
396,Vishal Mishra,None,Jab Koi Baat
397,Vishal Mishra,None,Tujhse Naraz Nahi
398,Vishal Mishra,None,Main Wahi Hoon


In [ ]:
df_ind.isna().sum()

,0
Artist,0
Country,400
Song,0


In [ ]:
## lets do the same for the usa artists
df["Country"].value_counts()

,count
Country,
USA,440
India,400


In [ ]:
df_us = df[df["Country"] == "USA"]
df_us

,Artist,Country
0,Taylor Swift,USA
1,Taylor Swift,USA
2,Taylor Swift,USA
3,Taylor Swift,USA
4,Taylor Swift,USA
...,...,...
435,Olivia Rodrigo,USA
436,Olivia Rodrigo,USA
437,Olivia Rodrigo,USA
438,Olivia Rodrigo,USA


In [ ]:
df_us["Artist"].unique(), len(df_us["Artist"].unique())

(array(['Taylor Swift', 'Eminem', 'Drake', 'Billie Eilish',
        'Ariana Grande', 'Rihanna', 'Justin Bieber', 'Bruno Mars',
        'Post Malone', 'Lady Gaga', 'Katy Perry', 'The Weeknd', 'Beyoncé',
        'Ed Sheeran', 'Sia', 'Imagine Dragons', 'Demi Lovato',
        'Nicki Minaj', 'Shawn Mendes', 'Dua Lipa', 'Coldplay',
        'Olivia Rodrigo'], dtype=object),
 22)

In [ ]:
len(df_us)

440

In [ ]:
df_us["Artist"].unique()

array(['Taylor Swift', 'Eminem', 'Drake', 'Billie Eilish',
       'Ariana Grande', 'Rihanna', 'Justin Bieber', 'Bruno Mars',
       'Post Malone', 'Lady Gaga', 'Katy Perry', 'The Weeknd', 'Beyoncé',
       'Ed Sheeran', 'Sia', 'Imagine Dragons', 'Demi Lovato',
       'Nicki Minaj', 'Shawn Mendes', 'Dua Lipa', 'Coldplay',
       'Olivia Rodrigo'], dtype=object)

In [ ]:
df_us = add_songs_to_dataframe(df_us,"/content/song name usa_1.txt")
df_us

,Artist,Country,Song
0,Taylor Swift,None,Fortnight
1,Taylor Swift,None,The Tortured Poets Department
2,Taylor Swift,None,My Boy Only Breaks His Favorite Toys
3,Taylor Swift,None,Down Bad
4,Taylor Swift,None,"So Long, London"
...,...,...,...
435,Olivia Rodrigo,None,Bye Bye Baby
436,Olivia Rodrigo,None,Don't Lose Your Head
437,Olivia Rodrigo,None,Still Love You
438,Olivia Rodrigo,None,Cinderella


In [ ]:
df_us[df_us["Artist"] == "Taylor Swift"]

,Artist,Country,Song
0,Taylor Swift,None,Fortnight
1,Taylor Swift,None,The Tortured Poets Department
2,Taylor Swift,None,My Boy Only Breaks His Favorite Toys
3,Taylor Swift,None,Down Bad
4,Taylor Swift,None,"So Long, London"
5,Taylor Swift,None,But Daddy I Love Him
6,Taylor Swift,None,Fresh Out the Slammer
7,Taylor Swift,None,Florida!!!
8,Taylor Swift,None,Guilty as Sin?
9,Taylor Swift,None,Who’s Afraid of Little Old Me?


In [ ]:
len(df_us["Artist"].value_counts())

22

In [ ]:
len(df_us)

440

In [ ]:
df_us["Country"].fillna("USA",inplace=True)
df_ind["Country"].fillna("India",inplace=True)

<ipython-input-91-94050eb09cfa>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_us["Country"].fillna("USA",inplace=True)
<ipython-input-91-94050eb09cfa>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

In [ ]:
df_us

,Artist,Country,Song
0,Taylor Swift,USA,Fortnight
1,Taylor Swift,USA,The Tortured Poets Department
2,Taylor Swift,USA,My Boy Only Breaks His Favorite Toys
3,Taylor Swift,USA,Down Bad
4,Taylor Swift,USA,"So Long, London"
...,...,...,...
435,Olivia Rodrigo,USA,Bye Bye Baby
436,Olivia Rodrigo,USA,Don't Lose Your Head
437,Olivia Rodrigo,USA,Still Love You
438,Olivia Rodrigo,USA,Cinderella


In [ ]:
df_ind

,Artist,Country,Song
0,Arijit Singh,India,Kesariya
1,Arijit Singh,India,Tum Hi Ho
2,Arijit Singh,India,Raabta
3,Arijit Singh,India,Channa Mereya
4,Arijit Singh,India,Pal
...,...,...,...
395,Vishal Mishra,India,Tum Hi Aana
396,Vishal Mishra,India,Jab Koi Baat
397,Vishal Mishra,India,Tujhse Naraz Nahi
398,Vishal Mishra,India,Main Wahi Hoon


In [ ]:
df_us.isna().sum()

,0
Artist,0
Country,0
Song,0


In [ ]:
df_ind.isna().sum()

,0
Artist,0
Country,0
Song,0


In [ ]:
#lets concate this both dataset now
df_2 = pd.concat([df_ind,df_us],ignore_index=True)
df_2.head()

,Artist,Country,Song
0,Arijit Singh,India,Kesariya
1,Arijit Singh,India,Tum Hi Ho
2,Arijit Singh,India,Raabta
3,Arijit Singh,India,Channa Mereya
4,Arijit Singh,India,Pal


In [ ]:
len(df_2)

840

In [ ]:
df_2.isna().sum()

,0
Artist,0
Country,0
Song,0


In [ ]:
def add_youtube_urls(df):
    # Create a new column 'url' and initialize it with None
    df['url'] = None

    # Iterate over each row to fetch the URL for each song
    for index, row in df.iterrows():
        song_name = row['Song']
        artist_name = row['Artist']
        url = search_youtube(song_name, artist_name)
        df.at[index, 'url'] = url  # Update the DataFrame with the fetched URL

    return df

In [ ]:
df_2  = add_youtube_urls(df_2)

In [ ]:
df_2.head()

,Artist,Country,Song,url
0,Arijit Singh,India,Kesariya,https://www.youtube.com/watch?v=BddP6PYo2gs
1,Arijit Singh,India,Tum Hi Ho,https://www.youtube.com/watch?v=Umqb9KENgmk
2,Arijit Singh,India,Raabta,https://www.youtube.com/watch?v=VWILvr01bxo
3,Arijit Singh,India,Channa Mereya,https://www.youtube.com/watch?v=bzSTpdcs-EI
4,Arijit Singh,India,Pal,https://www.youtube.com/watch?v=nDOpAEW2q2w


In [ ]:
df_2.isna().sum()

,0
Artist,0
Country,0
Song,0
url,0


In [ ]:
df_2[:5]

,Artist,Country,Song,url
0,Arijit Singh,India,Kesariya,https://www.youtube.com/watch?v=BddP6PYo2gs
1,Arijit Singh,India,Tum Hi Ho,https://www.youtube.com/watch?v=Umqb9KENgmk
2,Arijit Singh,India,Raabta,https://www.youtube.com/watch?v=VWILvr01bxo
3,Arijit Singh,India,Channa Mereya,https://www.youtube.com/watch?v=bzSTpdcs-EI
4,Arijit Singh,India,Pal,https://www.youtube.com/watch?v=nDOpAEW2q2w


In [ ]:
df_2[:1]

,Artist,Country,Song,url
0,Arijit Singh,India,Kesariya,https://www.youtube.com/watch?v=BddP6PYo2gs


In [ ]:
df_2[395:401]

,Artist,Country,Song,url
395,Vishal Mishra,India,Tum Hi Aana,https://www.youtube.com/watch?v=7HhnSpALS6U
396,Vishal Mishra,India,Jab Koi Baat,https://www.youtube.com/watch?v=rFU28HBP7B0
397,Vishal Mishra,India,Tujhse Naraz Nahi,https://www.youtube.com/watch?v=uyzUJwWg9yM
398,Vishal Mishra,India,Main Wahi Hoon,https://www.youtube.com/watch?v=K0ibsAQTyRQ
399,Vishal Mishra,India,Kahani Suno 2.0,https://www.youtube.com/watch?v=SJB6pWk77bg
400,Taylor Swift,USA,Fortnight,https://www.youtube.com/watch?v=q3zqJs7JUCQ


In [ ]:
download_youtube_audio(df_2[:1],output_folder="Try")

[1] Audio fetched successfully: 1.Kesariya_(Arijit_Singh) - https://www.youtube.com/watch?v=BddP6PYo2gs


In [ ]:
download_youtube_audio(df_2[:400], output_folder="audio_files_first_400_hindi")

[1] Audio fetched successfully: 1.Kesariya_(Arijit_Singh) - https://www.youtube.com/watch?v=BddP6PYo2gs
[2] Audio fetched successfully: 2.Tum Hi Ho_(Arijit_Singh) - https://www.youtube.com/watch?v=Umqb9KENgmk
[3] Audio fetched successfully: 3.Raabta_(Arijit_Singh) - https://www.youtube.com/watch?v=VWILvr01bxo
[4] Audio fetched successfully: 4.Channa Mereya_(Arijit_Singh) - https://www.youtube.com/watch?v=bzSTpdcs-EI
[5] Audio fetched successfully: 5.Pal_(Arijit_Singh) - https://www.youtube.com/watch?v=nDOpAEW2q2w
[6] Audio fetched successfully: 6.Hasi_(Arijit_Singh) - https://www.youtube.com/watch?v=oyaudgo5_8Y
[7] Audio fetched successfully: 7.Kabira_(Arijit_Singh) - https://www.youtube.com/watch?v=6UPZo8PX9OI
[8] Audio fetched successfully: 8.Agar Tum Saath Ho_(Arijit_Singh) - https://www.youtube.com/watch?v=fs7-8M1VbZU
[9] Audio fetched successfully: 9.Tujhe Kitna Chahne Lage_(Arijit_Singh) - https://www.youtube.com/watch?v=AgX2II9si7w
[10] Audio fetched successfully: 10.Shayad_(Ari

In [ ]:
import shutil
shutil.make_archive("/content/audio_files_first_400_hindi","zip","/content/audio_files_first_400_hindi")

'/content/audio_files_first_400_hindi.zip'

In [ ]:
download_youtube_audio(df_2[400:], output_folder="audio_files_last_440_english")

[1] Audio fetched successfully: 1.Fortnight_(Taylor_Swift) - https://www.youtube.com/watch?v=q3zqJs7JUCQ
[2] Audio fetched successfully: 2.The Tortured Poets Department_(Taylor_Swift) - https://www.youtube.com/watch?v=RQMz4JDbtmI
[3] Audio fetched successfully: 3.My Boy Only Breaks His Favorite Toys_(Taylor_Swift) - https://www.youtube.com/watch?v=wRKXAAV6jh4
[4] Audio fetched successfully: 4.Down Bad_(Taylor_Swift) - https://www.youtube.com/watch?v=EVbtjaWXQVg
[5] Audio fetched successfully: 5.So Long, London_(Taylor_Swift) - https://www.youtube.com/watch?v=CCUr2pNJft4
[6] Audio fetched successfully: 6.But Daddy I Love Him_(Taylor_Swift) - https://www.youtube.com/watch?v=U2W173hRfyA
[7] Audio fetched successfully: 7.Fresh Out the Slammer_(Taylor_Swift) - https://www.youtube.com/watch?v=0EKbEP2L32M
[8] Audio fetched successfully: 8.Florida!!!_(Taylor_Swift) - https://www.youtube.com/watch?v=uEssK8o3jKg
[9] Audio fetched successfully: 9.Guilty as Sin?_(Taylor_Swift) - https://www.youtub

In [ ]:
shutil.make_archive("/content/audio_files_last_440_english","zip","/content/audio_files_last_440_english")

'/content/audio_files_last_440_english.zip'

In [ ]:
df_2.to_csv("MY_MUSIC_URLS.csv",index=False)